## Overview

To facilitate graph analysis, we need to support sharing layouts between graphs, something I haven't seen done very well elsewhere.  In particular, it should be possible to:

* Use a consistent layout for multiple graphs:
    * &#x2713; Capture the layout of a graph $G_0$ that has been rendered, or
    * &#x2713; Create a layout without rendering $G_0$.
    * &#x2713; Render graph $G_1$ that has the same vertices as $G_0$, in the same order.
    * <strike>Render graph $G_1$ that has the same vertices as $G_0$, in a different order.</strike> The way we define vertex ids makes this impossible I think.
    * Render graph $G_1$ that has a subset of the vertices in $G_0$, in arbitrary order.
    * Render graph $G_1$ that has a superset of the vertices in $G_0$, in arbitrary order.
        * In this case the new vertices should be laid-out using the chosen layout algorithm, without disturbing the existing vertices.
* &#x2713; "Pin" a few key vertices while letting the layout handle the remaining vertices.
* &#x2713; An implicit requirement is the ability to perform a layout on some vertices while leaving others unchanged.
* Another implicit requirement is the ability to supply vertex ids *and* vertex coordinates to the layout process. 

So the process to layout graph $G_1$, given an (optional) layout of graph $G_0$ is (I think):

* Extract the edges for $G_1$.
    * Either an $E \times 2$ matrix, or
    * Two $|E|$ vectors.
* Identify the vertex ids $V_1$ induced by those edges.
* Merge optional user-supplied vertex ids with $V_1$ (to allow for disconnected vertices).
* Create a set of completely masked vertex coordinates, $V_{C_1}$.
* If the user supplied an "other" toyplot.layout.GraphLayout object from $G_0$, use the $V_0$ vertex ids to merge $V_{C_0}$ coordinates into $V_{C_1}$.
* If the user supplied a set of external vertex coordinates $V_{C_{external}}$ with length $|V_1|$, merge them into $V_{C_1}$.
* Apply a layout algorithm to any remaining masked values in $V_{C_1}$.

Allowed input formats:

* toyplot.graph(edges, [vids], [olayout], [vcoordinates])
* toyplot.graph(sources, targets, [vids], [olayout], [vcoordinates])

In [1]:
import numpy
import toyplot.color
import toyplot.generate

In [2]:
numpy.random.seed(1234)

In [3]:
# Random graph
#vertices = ["a", "b", "c", "d", "e", "f", "g", "h", "i", "j"]
#graph = numpy.random.choice(vertices, (40, 2))

full_graph = toyplot.generate.prufer_tree(numpy.random.choice(4, 12))
partial_graph = full_graph[:-3]

# Reingold tree
#graph = numpy.array([[0,1],[0,2],[1,3],[2,4],[2,5],[3,6],[4,7],[5,8],[5,9],[6,10],[7,11],[9,12],[10,13],[12,14]])

In [4]:
%%time

colormap = toyplot.color.LinearMap(toyplot.color.Palette(["white", "yellow", "red"]))

full_layout = toyplot.layout.graph(full_graph)

vcoordinates = numpy.ma.masked_all((full_layout.vcount, 2))
vcoordinates[0] = (-1, 0)
vcoordinates[1] = (0, 0)
vcoordinates[2] = (1, 0)

canvas = toyplot.Canvas(width=1000, height=500)
axes = canvas.axes(show=False)
axes.aspect = "expand-domain"
mark = axes.graph(
    full_graph,
    #vcoordinates = vcoordinates,
    vcolor=colormap,
    vmarker="o",
    vstyle={"stroke":"black"},
    vsize=20,
    ecolor="black",
    eopacity=0.2,
);

vcoordinates = mark.vcoordinates.copy()
axes.text(vcoordinates.T[0], vcoordinates.T[1], mark.vid, color="black")

INFO:toyplot:Graph layout time: 71.4020729065 ms
INFO:toyplot:Graph layout time: 51.6111850739 ms


CPU times: user 130 ms, sys: 6.25 ms, total: 136 ms
Wall time: 133 ms


In [5]:
%%time

colormap = toyplot.color.LinearMap(toyplot.color.Palette(["white", "yellow", "red"]))

partial_layout = toyplot.layout.graph(partial_graph)

canvas = toyplot.Canvas(width=1000, height=500)
axes = canvas.axes(show=False)
axes.aspect = "expand-domain"
mark = axes.graph(
    partial_graph,
    olayout=full_layout,
    vcolor=colormap,
    vmarker="o",
    vstyle={"stroke":"black"},
    vsize=20,
    ecolor="black",
    eopacity=0.2,
);

vcoordinates = mark.vcoordinates.copy()
axes.text(vcoordinates.T[0], vcoordinates.T[1], mark.vid, color="black")

INFO:toyplot:Graph layout time: 51.0020256042 ms
INFO:toyplot:Graph layout time: 49.3779182434 ms


CPU times: user 110 ms, sys: 1.48 ms, total: 112 ms
Wall time: 110 ms
